Every month, my project recieves a large number of invoices. Our business team spends countless hours going through each invoice to categorize these expenses and share with stakeholders how much money is being charged to each category. For one particular company (SOS), I built a NLP model that successfully catorgorized 100% of their expenses over the last 7 months. For confidetiality purposes, the .csv file used to train the data and the results of the project cannot be shown. However, the below code shows the process of building the model.

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('invoice_data.csv',encoding='windows-1252')

In [ ]:
sample_data = data.head(300)

Feature Engineering - unbalanced categories in the training set

In [ ]:
sample_data.groupby('item_type').count()

,Description
item_type,
Admin,86
Asbuild,20
Contol,1
Control,75
Layout,41
Quality,64
Safety,5
Tax,4


In [ ]:
add = sample_data[sample_data['item_type'] == 'Admin'][0:30]

In [ ]:
add1 = sample_data[sample_data['item_type'] == 'Control'][0:30]

In [ ]:
add2 = sample_data[sample_data['item_type'] == 'Quality'][0:30]

In [ ]:
add3 = sample_data[sample_data['item_type'] == 'Layout'][0:30]

In [ ]:
sample_data = sample_data[sample_data['item_type'] != 'Admin']

In [ ]:
sample_data = sample_data[sample_data['item_type'] != 'Control']

In [ ]:
sample_data = sample_data[sample_data['item_type'] != 'Quality']

In [ ]:
sample_data = sample_data[sample_data['item_type'] != 'Layout']

In [ ]:
sample_data = sample_data.append(add)

<ipython-input-255-a1ae89528bca>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_data = sample_data.append(add)


In [ ]:
sample_data = sample_data.append(add1)

<ipython-input-256-10bc0753b00f>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_data = sample_data.append(add1)


In [ ]:
sample_data = sample_data.append(add2)

<ipython-input-257-fabfdc764221>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_data = sample_data.append(add2)


In [ ]:
sample_data = sample_data.append(add3)

<ipython-input-258-242cb1bf4e01>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_data = sample_data.append(add3)


In [ ]:
sample_data.groupby('item_type').count()

,Description
item_type,
Admin,30
Asbuild,20
Contol,1
Control,30
Layout,30
Quality,30
Safety,5
Tax,4


In [ ]:
sample_data = sample_data.dropna()

Building the NLP model & vectorizing the training set

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(nltk.corpus.stopwords.words('english'))
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')

In [ ]:
def preprocess(text):
    words = tokenizer.tokenize(text.lower())
    words = [w for w in words if not w in stop_words]
    return ' '.join(words)

In [ ]:
sample_data['processed_text'] = sample_data['Description'].apply(preprocess)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_data['processed_text'], sample_data['item_type'], test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)

MultinomialNB()

In [ ]:
y_pred = clf.predict(X_test_vectorized)

In [ ]:
test_df = pd.DataFrame({'description': X_test, 'actual_item_type': y_test, 'predicted_item_type': y_pred})
test_df.to_csv('invoice_data_testing_set.csv', index=False)

In [ ]:
import pandas as pd
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
new_data = pd.read_csv('test.csv')

In [ ]:
X_new = vectorizer.transform(new_data['Description'])

In [ ]:
X_new = X_new.toarray()
X_new = [[word for word in row if word not in stop_words] for row in X_new]

In [ ]:
predictions = clf.predict(X_new)

In [ ]:
new_df = pd.DataFrame({'description':new_data['Description'],'predictions':predictions})

In [ ]:
new_df.to_csv('classification.csv', index=False)

In [ ]:
filename = 'nlp_SOS_model.sav'
pickle.dump(clf, open(filename, 'wb'))